In [1]:
import _pickle as pickle
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import copy
import glob
import gc

In [2]:
full_df = []

exp_list = glob.glob('/data/Bodenreider_UMLS_DL/Interns/Bernal/UMLS2020AB_sapbert.2000-NN_DataFrame.p')

print(exp_list)

for filename in tqdm(exp_list):
    file = open(filename,'rb')
    full_df.append(pickle.load(file))
    file.close()

['/data/Bodenreider_UMLS_DL/Interns/Bernal/UMLS2020AB_sapbert.2000-NN_DataFrame.p']


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [11:48<00:00, 708.62s/it]


In [3]:
umls2020AB_df = full_df[0]

In [4]:
umls2020AB_df.describe()

,0,num_syms
count,430135.000000,430135.000000
mean,11.463557,7.497516
std,9.860294,35.940232
min,3.000000,0.000000
25%,7.000000,0.000000
50%,10.000000,0.000000
75%,13.000000,4.000000
max,1104.000000,550.000000


In [10]:
pd.set_option('max_colwidth',500)

In [11]:
umls2020AB_df[umls2020AB_df.strings == 'Dandie Dinmont Terrier']

,0,strings,auis,2020AA_synonyms,synonym_strings,num_syms,sapbert_2000-NN_strings,sapbert_2000-NN_auis,sapbert_2000-NN_dist,sapbert_2000-NN_recall
8538331,10,Dandie Dinmont Terrier,A31599751,"[A3013170, A16965279, A10802552, A16966835]","[Dandie dinmont terrier, Dandie Dinmont terrier, Dandie Dinmont Terrier, Dandie Dinmont terrier (organism)]",4,"[Dandie Dinmont Terrier, Dandie dinmont terrier, Dandie Dinmont terrier, Dandie Dinmont terrier (organism), Cairn terrier, Cairn Terrier, Soft-coated wheaten terrier, Soft Coated Wheaten Terrier, Kerry blue terrier (organism), Leuciscus dandia, Airedale Terrier, Airedale terrier, Kerry blue terrier, Kerry Blue Terrier, Welsh Terrier, Welsh terrier, Bedlington Terrier, Bedlington terrier, Norfolk terrier, Norfolk Terrier, Cairn terrier (organism), Lakeland terrier, Lakeland Terrier, Airedale ...","[A10802552, A3013170, A16965279, A16966835, A3003023, A10799913, A3072011, A10817400, A3528362, A26734254, A10815951, A2990092, A3036403, A10808683, A10795376, A3083376, A10797631, A2995811, A3049822, A10812519, A3335229, A3037299, A10808946, A3531711, A3586817, A3711569, A3785888, A3805910, A3070871, A10817234, A3001100, A10798248, A3621242, A10795439, A3083757, A3050007, A10812534, A30394316, A3899982, A24407930, A10793776, A3077220, A3531869, A27966991, A17566466, A3326274, A10816961, A30...","[0.0, 0.0, 0.0, 8.009628, 137.91754, 137.91754, 139.01683, 139.13464, 139.8493, 140.57378, 140.86127, 140.86127, 140.87146, 140.87146, 140.96123, 140.96123, 143.03625, 143.03625, 144.10829, 144.10829, 144.1291, 144.19017, 144.19017, 144.70679, 145.59254, 146.11206, 147.30128, 147.38994, 147.58711, 147.58711, 147.90125, 147.90125, 148.6371, 149.38478, 149.38478, 150.11333, 150.11333, 150.48326, 150.9384, 151.37096, 151.67645, 151.67645, 151.95583, 152.5951, 152.66243, 153.59464, 154.25658, 15...","[0.25, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"


In [12]:
#Creating Synonym Edge List

edges = []

for i,row in tqdm(umls2020AB_df.iterrows(),total=len(umls2020AB_df)):
    
    string = row['strings']
    
    synonyms = set(row['synonym_strings'])
    predicted_synonyms = row['sapbert_2000-NN_strings'][:200]
    predicted_synonym_dist = row['sapbert_2000-NN_dist'][:200]
    
    processed_preds = set()
    
    for pred, dist in zip(predicted_synonyms, predicted_synonym_dist):
        
        if pred not in processed_preds:
            if pred in synonyms:
                edges.append((string, pred, 1, dist))
            else:
                edges.append((string, pred, 0, dist))
            
        processed_preds.add(pred)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 430135/430135 [02:08<00:00, 3337.56it/s]


In [ ]:
# pickle.dump(edges, open('/data/Bodenreider_UMLS_DL/Interns/Bernal/umls2020AB_sapbert_edges.p','wb'))

In [ ]:
edges[-10:]

In [ ]:
testing_data_df = pd.DataFrame(edges)

validation_set = []
testing_set = []

for i,g in testing_data_df.groupby(2):
    
    perm = g.sample(len(g), random_state=np.random.RandomState(42))
    
    val = perm[:int(len(g)*0.01)]
    test = perm[int(len(g)*0.01):]
    
    validation_set.append(val)
    testing_set.append(test)

validation_set = pd.concat(validation_set)
testing_set = pd.concat(testing_set)

subset = [(h,t,l) for h,t,l in zip(validation_set[0],validation_set[1],validation_set[2])]

In [ ]:
len(subset)

In [ ]:
pickle.dump(subset, open('/data/Bodenreider_UMLS_DL/Interns/Bernal/umls2020AB_sapbert_edges_subset.p','wb'))

In [ ]:
edges = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Bernal/uva_test_subset.p','rb'))

In [ ]:
glob.glob('/data/Bodenreider_UMLS_DL/Interns/Bernal/*predictions')

In [ ]:
pred_filename = '/data/Bodenreider_UMLS_DL/Interns/Bernal/ubert_mlm_uva_test_290020_2_predictions/umls2020_ubert_mlm_uva_test_290020_2_predictions_{}.p'

# pred_filename = '/data/Bodenreider_UMLS_DL/Interns/Bernal/sapbert_ubert_uva_test_1392096_predictions/umls2020_sapbert_ubert_uva_test_1392096_predictions_{}.p'

preds = []
i=0

while os.path.exists(pred_filename.format(i)):
    f = open(pred_filename.format(i),'rb')
    preds.append(pickle.load(f))
    i+=1
    
preds = np.vstack(preds)

In [ ]:
len(edges),len(preds)/2

In [ ]:
edges_to_test = edges[:int(len(preds)/2)]

In [ ]:
forward_preds = []
backward_preds = []

for i,p in enumerate(preds):
    if i % 2 == 0:
        forward_preds.append(p)
    else:
        backward_preds.append(p)

In [ ]:
edges_to_test_df = pd.DataFrame(edges_to_test)

In [ ]:
edges_to_test_df['forward_pred'] = np.argmax(np.vstack(forward_preds),axis=1)
edges_to_test_df['backward_pred'] = np.argmax(np.vstack(backward_preds),axis=1)

edges_to_test_df['forward_pred'] = [not(e) for e in edges_to_test_df['forward_pred']]
edges_to_test_df['backward_pred'] = [not(e) for e in edges_to_test_df['backward_pred']]

In [ ]:
edges_to_test_df['forward_pred'] = np.argmax(np.vstack(forward_preds),axis=1)
edges_to_test_df['backward_pred'] = np.argmax(np.vstack(backward_preds),axis=1)

In [ ]:
pd.set_option('max_colwidth',100)

In [ ]:
edges_to_test_df[:50]

In [ ]:
edges_to_test_df.groupby('forward_pred').count()

In [ ]:
edges_to_test_df.groupby('backward_pred').count()

In [ ]:
edges_to_test_df.groupby(2).count()

In [ ]:
labels = edges_to_test_df[2]

forward_f1 = metrics.precision_recall_fscore_support(labels, edges_to_test_df['forward_pred'], pos_label=0, average='binary')
backward_f1 = metrics.precision_recall_fscore_support(labels, edges_to_test_df['backward_pred'], pos_label=0, average='binary')

forward_f1, backward_f1

In [ ]:
validation_set = []
testing_set = []

for i,g in edges_to_test_df.groupby(2):
    
    perm = g.sample(len(g), random_state=np.random.RandomState(42))
    
    val = perm[:int(len(g)*0.02)]
    test = perm[int(len(g)*0.02):]
    
    validation_set.append(val)
    testing_set.append(test)

In [ ]:
validation_set = pd.concat(validation_set)
testing_set = pd.concat(testing_set)

In [ ]:
len(testing_set), len(validation_set)

In [ ]:
from sklearn import metrics

In [ ]:
scores = []

for threshold in tqdm(np.arange(min(validation_set[3]),max(validation_set[3]))):
    labels = validation_set[2]
    preds = [d < threshold for d in validation_set[3]]
    
    f1 = metrics.f1_score(labels, preds)
    scores.append(f1)

In [ ]:
thresholds = np.arange(min(validation_set[3]),max(validation_set[3]))
plt = pd.DataFrame(thresholds)
plt['scores'] = scores

optimal_thresh = thresholds[np.argmax(scores)]
plt.plot(0,'scores')

In [ ]:
labels = validation_set[2]
sapbert_preds = [d < optimal_thresh for d in validation_set[3]]
    
sapbert_f1 = metrics.f1_score(labels, sapbert_preds)
forward_ubert_f1 = metrics.f1_score(labels, validation_set['forward_pred'])
backward_ubert_f1 = metrics.f1_score(labels, validation_set['backward_pred'])

In [ ]:
sapbert_f1

In [ ]:
forward_ubert_f1

In [ ]:
backward_ubert_f1